In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

try:
    data = pd.read_csv('Euphoria.csv')
except FileNotFoundError:
    print("File not found. Please check the filename and path.")
    exit()

data.info()
print(data.head())

X = data.drop(columns=['Name'])
y = data['Name']


X = X + np.random.normal(0, 1, X.shape)

label_encoder = LabelEncoder()

label_encoder.fit(y)

y_encoded = label_encoder.transform(y)

num_classes = len(label_encoder.classes_)
y_categorical = to_categorical(y_encoded)

X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = Sequential([
    Dense(16, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train_scaled, y_train, epochs=30, batch_size=32, validation_split=0.1)

loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f'Test Accuracy: {accuracy*100:.2f}%')

model.save('tree_species_model.h5')

import pickle
with open('label_encoder.pkl', 'wb') as le_file:
    pickle.dump(label_encoder, le_file)
with open('scaler.pkl', 'wb') as sc_file:
    pickle.dump(scaler, sc_file)